In [13]:
import Chest_x_ray
import torch
from pathlib import Path

In [2]:
def load_model(model_path):
    model = Chest_x_ray.ConvolutionalNetwork()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

In [6]:
best_model_checkpoint = "checkpoints_xrays/train_loss_0.49879632698246307_val_loss_0.6052662283182144.pt"
model = load_model(best_model_checkpoint)

In [8]:
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1032256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=2, bias=True)
)

# Loading Test data

In [14]:
path = Path("/data/analytics/naveen.bansal/Jeremy/data/chest_xray/")
test_data_path = path/'test'
test_imgs,test_labels = Chest_x_ray.load_data(test_data_path)
test_labels = Chest_x_ray.map_labels(test_labels)
test_dataset=Chest_x_ray.CNN_Dataset(test_imgs, test_labels)
test_dataloader= torch.utils.data.DataLoader(test_dataset, batch_size = 5,shuffle=True)

In [15]:
def model_metric(li_y_true,li_y_pred):   
    TP=0
    TN=0
    FP=0
    FN=0
    li=zip(li_y_true,li_y_pred)
    for item in li:
        true=item[0]
        pred=item[1]
        ground_truth=[it[0] for it in true]
        prediction=[it[0] for it in pred]
        for g,p in zip(prediction,ground_truth):
            if g==1 and p==1:
                TP=TP+1
            if g==0 and p==1:
                FP=FP+1
            if g==1 and p==0:
                FN=FN+1
            if g==0 and p==0:
                TN=TN+1
    accuracy=(TP+TN)/len(test_labels)
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1score=2*((precision*recall)/(precision+recall))
    return accuracy, recall,precision,F1score

In [18]:
import pdb
li_y_pred=[]
li_y_true=[]

if torch.cuda.is_available():
    model = model.cuda()

for batch in test_dataloader:
        x=batch[0]
        y_true=batch[1]
               
        if torch.cuda.is_available():
            x = x.cuda()
            y_true  = y_true.cuda()
            
        y_pred=model.forward(x)
        y_pred=torch.argmax(y_pred,dim=1).reshape(-1,1).tolist()
        y_true=y_true.tolist()
        li_y_true.append(y_true)
        li_y_pred.append(y_pred)    
accuracy, recall,precision,F1score=model_metric(li_y_true,li_y_pred)
print (f" Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1 score: {F1score}")

 Accuracy: 0.6987179487179487, Recall: 0.6753472222222222, Precision: 0.9974358974358974, F1 score: 0.805383022774327


In [55]:
from PIL import Image
from torchvision import datasets,transforms
import pdb

def predict(model,img_path):
    
    img = Image.open(img_path)
    trans=transforms.Compose([transforms.Resize(1024), 
                                  transforms.CenterCrop(1023), 
                                  transforms.Grayscale(num_output_channels=1), 
                                  transforms.ToTensor()])
    x=trans(img)
    if torch.cuda.is_available():
        x= x[None,:,:,:].cuda() 
    pred = model.forward(x)
    predicted_class = torch.argmax(pred).item()
    if predicted_class==0:
        predicted_class =  "Normal"
    else:
        predicted_class =  "Pneumonia"
    
    return predicted_class
    
    
    

In [57]:
prediction=predict(model, "/data/analytics/naveen.bansal/Jeremy/data/chest_xray/test/PNEUMONIA/person10_virus_35.jpeg")
print (f"Predicted Class: {prediction}")

Predicted Class: Pneumonia
